# Serializacja, walidacja i bazy danych w Pythonie

Notebook prowadzi przez techniki serializacji, walidacji oraz pracy z bazą danych.


## Cele
- utrwalić podstawy serializacji w formatach JSON, Pickle i CSV
- przećwiczyć pracę z bazą SQLite oraz ORM-ami (Pony ORM, SQLAlchemy)
- zrozumieć rolę Pydantic w walidacji i serializacji danych


## 1. Wprowadzenie
- **Serializacja**: zamiana obiektu Pythona na format tekstowy/binarny do przechowywania lub przesyłania.
- **Walidacja**: upewnienie się, że dane spełniają wymagany schemat.
- **Trwałość**: zapis danych poza pamięcią procesu (pliki, bazy danych).
- Wspólne formaty: JSON (tekstowy, interoperacyjny), Pickle (binarny specyficzny dla Pythona), CSV (prostota dla tabel).
- Bazy danych stosujemy, gdy potrzebujemy zapytań, relacji, równoległego dostępu i transakcji.


**Podsumowanie:** Serializacja i bazy to fundamenty pracy z danymi; walidacja pilnuje jakości informacji przed utrwaleniem.

**Pytanie kontrolne:** Kiedy warto przenieść dane z pliku JSON do bazy SQL?


## 2. Serializacja w Pythonie


### 2.1 JSON: podstawy i typy niestandardowe
`json` obsługuje typy proste (słowniki, listy, liczby, łańcuchy, `bool`, `None`).
Dla typów niestandardowych (np. `datetime`) używamy `default` lub własnego enkodera.


In [ ]:
import json
from datetime import datetime

user = {
    "name": "Ada",
    "age": 28,
    "registered_at": datetime(2023, 5, 4, 12, 30),
    "tags": ["python", "mentor"],
}

# json.dumps -> serializacja do łańcucha, default=str zapewnia obsługę datetime
payload = json.dumps(user, default=str, ensure_ascii=False, indent=2)
print(payload)

# json.loads -> odczyt do struktur Pythona
parsed = json.loads(payload)
print("Typ daty po odczycie:", type(parsed["registered_at"]))


#### Własny enkoder JSON
Możemy rozszerzyć serializację przez podklasę `json.JSONEncoder`.


In [ ]:
class DateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()  # konwersja do standardu ISO 8601
        return super().default(obj)

encoded = json.dumps(user, cls=DateTimeEncoder, indent=2, ensure_ascii=False)
print(encoded)


### 2.2 Pickle: serializacja dowolnych obiektów
`pickle` zapisuje obiekty Pythona w formacie binarnym. Jest szybki, ale niebezpieczny dla niezaufanych danych.


In [ ]:
import pickle

settings = {
    "feature_flag": True,
    "callback": lambda x: x * 2,  # funkcje również mogą być serializowane
}

# dumps -> bajty w pamięci
blob = pickle.dumps(settings)
print("Rozmiar bajtów:", len(blob))

# loads -> odtworzenie obiektu
loaded = pickle.loads(blob)
print(loaded["callback"](5))
print("UWAGA: nigdy nie ładuj pickle z niezaufanego źródła!")


### 2.3 CSV: format tabelaryczny
Dla prostych tabel używamy `csv.DictWriter` i `csv.DictReader`.


In [ ]:
import csv
from pathlib import Path

users = [
    {"name": "Adam", "email": "adam@example.com", "age": 30},
    {"name": "Beata", "email": "beata@example.com", "age": 24},
]

csv_path = Path("users.csv")

# Zapis danych do pliku CSV
with csv_path.open("w", newline="", encoding="utf-8") as handle:
    writer = csv.DictWriter(handle, fieldnames=["name", "email", "age"])
    writer.writeheader()
    writer.writerows(users)

# Odczyt danych z pliku CSV
with csv_path.open("r", newline="", encoding="utf-8") as handle:
    reader = csv.DictReader(handle)
    loaded_rows = list(reader)

print(loaded_rows)


**Podsumowanie:** JSON służy do interoperacyjnej wymiany, Pickle przechowuje skomplikowane obiekty, a CSV jest zwięzły dla tabel.

**Pytanie kontrolne:** Który format wybierzesz do wymiany danych między usługami HTTP i dlaczego?


### 🧩 Zadanie 1
Zapisz listę użytkowników (słowniki z polami `name`, `email`, `age`) do JSON i CSV, odczytaj je ponownie i policz średni wiek.


In [ ]:
# Rozwiązanie Zadania 1
import json
import csv
from pathlib import Path

people = [
    {"name": "Ania", "email": "ania@example.com", "age": 26},
    {"name": "Kuba", "email": "kuba@example.com", "age": 34},
    {"name": "Ola", "email": "ola@example.com", "age": 29},
]

json_file = Path("zad1_users.json")
csv_file = Path("zad1_users.csv")

# Zapis JSON
json_file.write_text(json.dumps(people, indent=2, ensure_ascii=False), encoding="utf-8")

# Zapis CSV
with csv_file.open("w", newline="", encoding="utf-8") as handle:
    writer = csv.DictWriter(handle, fieldnames=["name", "email", "age"])
    writer.writeheader()
    writer.writerows(people)

# Odczyt JSON
data_json = json.loads(json_file.read_text(encoding="utf-8"))

# Odczyt CSV
data_csv = []
with csv_file.open("r", newline="", encoding="utf-8") as handle:
    reader = csv.DictReader(handle)
    for row in reader:
        row["age"] = int(row["age"])  # konwersja tekstu na int
        data_csv.append(row)

# Średni wiek (na podstawie danych JSON)
avg_age = sum(item["age"] for item in data_json) / len(data_json)
print(f"Średni wiek: {avg_age:.1f}")


## 3. Bazy danych – niski poziom (sqlite3)
`sqlite3` pozwala tworzyć lokalne bazy SQL bez serwera. Używamy połączenia, kursora i zapytań z parametrami (`?`).


In [ ]:
import sqlite3
from pathlib import Path

db_path = Path("users.db")
db_path.unlink(missing_ok=True)  # czyszczenie poprzedniej bazy

with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute(
        """
        CREATE TABLE users (
            name TEXT,
            age INTEGER,
            email TEXT UNIQUE
        )
        """
    )
    cursor.executemany(
        "INSERT INTO users VALUES (?, ?, ?)",
        [
            ("Adam", 30, "adam@example.com"),
            ("Beata", 25, "beata@example.com"),
            ("Celina", 35, "celina@example.com"),
        ],
    )
    conn.commit()  # zatwierdzamy transakcję

with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name, age FROM users WHERE age >= ?", (30,))
    print(cursor.fetchall())


**Podsumowanie:** `sqlite3` daje pełną kontrolę nad SQL, ale wymaga ręcznego mapowania rekordów na obiekty.

**Pytanie kontrolne:** Po co używamy `?` w zapytaniach SQL zamiast wstrzykiwać wartości przez f-string?


### 🧩 Zadanie 2
Utwórz bazę `library.db`, tabela `books(title, author, year)`, dodaj kilka rekordów i wypisz książki z ostatnich pięciu lat.


In [ ]:
# Rozwiązanie Zadania 2
import sqlite3
from pathlib import Path
from datetime import datetime

current_year = datetime.now().year
lib_path = Path("library.db")
lib_path.unlink(missing_ok=True)

with sqlite3.connect(lib_path) as conn:
    cur = conn.cursor()
    cur.execute(
        """
        CREATE TABLE books (
            title TEXT,
            author TEXT,
            year INTEGER
        )
        """
    )
    cur.executemany(
        "INSERT INTO books VALUES (?, ?, ?)",
        [
            ("Python Tricks", "Dan Bader", current_year - 1),
            ("Clean Code", "Robert C. Martin", 2008),
            ("Effective Python", "Brett Slatkin", current_year - 3),
        ],
    )
    conn.commit()

with sqlite3.connect(lib_path) as conn:
    cur = conn.cursor()
    cur.execute(
        "SELECT title, author, year FROM books WHERE year >= ?",
        (current_year - 5,),
    )
    print(cur.fetchall())


## 4. ORM-y: Pony ORM i SQLAlchemy
ORM mapuje tabele na klasy i pozwala pracować na obiektach zamiast na SQL.


### 4.1 Pony ORM: konfiguracja i CRUD


In [ ]:
from pony.orm import Database, Required, db_session, select

pony_db = Database()

class PonyUser(pony_db.Entity):
    name = Required(str)
    age = Required(int)
    email = Required(str)

pony_db.bind(provider="sqlite", filename=":memory:")
pony_db.generate_mapping(create_tables=True)

with db_session:
    PonyUser(name="Adam", age=28, email="adam@example.com")
    PonyUser(name="Beata", age=34, email="beata@example.com")

with db_session:
    seniors = select(u for u in PonyUser if u.age > 30)[:]
    print([(user.name, user.age) for user in seniors])


### 4.2 SQLAlchemy: modele i relacje One-to-Many


In [ ]:
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine
from sqlalchemy.orm import declarative_base, relationship, Session

engine = create_engine("sqlite+pysqlite:///:memory:", echo=False)
Base = declarative_base()

class Department(Base):
    __tablename__ = "departments"
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    employees = relationship("Employee", back_populates="department")

class Employee(Base):
    __tablename__ = "employees"
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    salary = Column(Integer, nullable=False)
    department_id = Column(Integer, ForeignKey("departments.id"))
    department = relationship("Department", back_populates="employees")

Base.metadata.create_all(engine)

with Session(engine) as session:
    dev = Department(name="Development")
    session.add_all(
        [
            Employee(name="Ania", salary=6000, department=dev),
            Employee(name="Kuba", salary=4500, department=dev),
        ]
    )
    session.commit()

with Session(engine) as session:
    high_paid = session.query(Employee).filter(Employee.salary > 5000).all()
    print([(emp.name, emp.salary) for emp in high_paid])


**Podsumowanie:** ORM-y automatyzują konwersję danych między SQL a obiektami, upraszczając logikę aplikacji.

**Pytanie kontrolne:** Kiedy warto pozostać przy `sqlite3` zamiast sięgać po ORM?


### 🧩 Zadanie 3
Zdefiniuj model `Employee` w SQLAlchemy z polami `id`, `name`, `salary`, dodaj 3 rekordy i wypisz pracowników o pensji powyżej 5000.


In [ ]:
# Rozwiązanie Zadania 3
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import declarative_base, Session

sa_engine = create_engine("sqlite+pysqlite:///:memory:", echo=False)
SA_Base = declarative_base()

class EmployeeModel(SA_Base):
    __tablename__ = "employees_task"
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    salary = Column(Integer, nullable=False)

SA_Base.metadata.create_all(sa_engine)

with Session(sa_engine) as session:
    session.add_all(
        [
            EmployeeModel(name="Adam", salary=4800),
            EmployeeModel(name="Beata", salary=5200),
            EmployeeModel(name="Celina", salary=6500),
        ]
    )
    session.commit()

with Session(sa_engine) as session:
    rich = session.query(EmployeeModel).filter(EmployeeModel.salary > 5000).all()
    print([(emp.name, emp.salary) for emp in rich])


## 5. Walidacja i serializacja z Pydantic
Pydantic automatycznie waliduje dane na podstawie adnotacji typów i wspiera integrację z ORM.


In [ ]:
from pydantic import BaseModel, EmailStr, ValidationError, Field
from typing import List
from datetime import datetime

class Profile(BaseModel):
    bio: str = Field(max_length=140)
    tags: List[str]

class UserModel(BaseModel):
    name: str
    email: EmailStr
    age: int
    joined_at: datetime | None = None
    profile: Profile | None = None

payload = {
    "name": "Ada",
    "email": "ada@example.com",
    "age": 28,
    "joined_at": "2023-05-05T10:00:00",
    "profile": {"bio": "Mentorka Pythona", "tags": ["python", "mentor"]},
}

user = UserModel(**payload)
print(user.dict())

invalid_payload = {"name": "Beata", "email": "wrong", "age": "trzydzieści"}
try:
    UserModel(**invalid_payload)
except ValidationError as exc:
    print(exc)


### Integracja z ORM (`orm_mode`)


In [ ]:
from pydantic import BaseModel
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import declarative_base, Session

orm_engine = create_engine("sqlite+pysqlite:///:memory:")
OrmBase = declarative_base()

class OrmUser(OrmBase):
    __tablename__ = "orm_users"
    id = Column(Integer, primary key=True)
    name = Column(String)
    email = Column(String)

OrmBase.metadata.create_all(orm_engine)
with Session(orm_engine) as session:
    session.add_all([
        OrmUser(name="Ania", email="ania@example.com"),
        OrmUser(name="Kuba", email="kuba@example.com"),
    ])
    session.commit()

class OrmUserModel(BaseModel):
    id: int
    name: str
    email: EmailStr

    class Config:
        orm_mode = True

with Session(orm_engine) as session:
    users = session.query(OrmUser).all()
    payload = [OrmUserModel.from_orm(u).dict() for u in users]
    print(payload)


**Podsumowanie:** Pydantic zapewnia walidację i serializację, a tryb `orm_mode` pozwala łatwo mapować obiekty ORM.

**Pytanie kontrolne:** Co się stanie, gdy Pydantic dostanie dane niezgodne z typem?


### 🧩 Zadanie 4
Utwórz model `UserModel` (Pydantic) z polami `name`, `email`, `age`. Zwaliduj dane z JSON-a i zapisz poprawnych użytkowników do SQLite.


In [ ]:
# Rozwiązanie Zadania 4
import json
import sqlite3
from pathlib import Path
from pydantic import BaseModel, EmailStr, ValidationError

class SimpleUser(BaseModel):
    name: str
    email: EmailStr
    age: int

json_payload = json.dumps([
    {"name": "Adam", "email": "adam@example.com", "age": 32},
    {"name": "Beata", "email": "invalid", "age": 29},
])

records = json.loads(json_payload)
validated: list[SimpleUser] = []
for record in records:
    try:
        validated.append(SimpleUser(**record))
    except ValidationError as exc:
        print("Błędny rekord:", record, exc)

db_file = Path("zad4_users.db")
db_file.unlink(missing_ok=True)

with sqlite3.connect(db_file) as conn:
    cur = conn.cursor()
    cur.execute("CREATE TABLE users (name TEXT, email TEXT, age INTEGER)")
    cur.executemany(
        "INSERT INTO users VALUES (?, ?, ?)",
        [(u.name, u.email, u.age) for u in validated],
    )
    conn.commit()

with sqlite3.connect(db_file) as conn:
    cur = conn.cursor()
    cur.execute("SELECT * FROM users")
    print(cur.fetchall())


## 6. Połączenie: ORM + Pydantic + JSON
Tworzymy pipeline: baza → Pydantic → JSON oraz JSON → Pydantic → baza.


In [ ]:
from pony.orm import Database, Required, db_session, select
from pydantic import BaseModel, EmailStr
import json
from pathlib import Path

combo_db = Database()

class ComboUser(combo_db.Entity):
    name = Required(str)
    email = Required(str)
    age = Required(int)

combo_db.bind(provider="sqlite", filename=":memory:")
combo_db.generate_mapping(create_tables=True)

with db_session:
    ComboUser(name="Ania", email="ania@example.com", age=25)
    ComboUser(name="Kuba", email="kuba@example.com", age=31)

class ComboUserModel(BaseModel):
    name: str
    email: EmailStr
    age: int

    class Config:
        orm_mode = True

with db_session:
    users = select(u for u in ComboUser)[:]
    validated = [ComboUserModel.from_orm(u) for u in users]

json_path = Path("combo_users.json")
json_path.write_text(
    json.dumps([user.dict() for user in validated], indent=2, ensure_ascii=False),
    encoding="utf-8",
)
print(json_path.read_text(encoding="utf-8"))


**Podsumowanie:** Walidacja przed serializacją do JSON chroni przed błędnymi danymi i ułatwia integrację.

**Pytanie kontrolne:** Dlaczego warto walidować dane wychodzące z bazy przed zapisaniem do pliku?


### 🧩 Zadanie 5
Pobierz wszystkich użytkowników z Pony ORM, przekształć ich w modele Pydantic i zapisz do pliku `users.json`.


In [ ]:
# Rozwiązanie Zadania 5
from pony.orm import Database, Required, db_session, select
from pydantic import BaseModel, EmailStr
import json
from pathlib import Path

user_db = Database()

class UserEntity(user_db.Entity):
    name = Required(str)
    email = Required(str)
    age = Required(int)

user_db.bind(provider="sqlite", filename=":memory:")
user_db.generate_mapping(create_tables=True)

with db_session:
    UserEntity(name="Adam", email="adam@example.com", age=30)
    UserEntity(name="Beata", email="beata@example.com", age=27)

class UserSchema(BaseModel):
    name: str
    email: EmailStr
    age: int

    class Config:
        orm_mode = True

with db_session:
    users = select(u for u in UserEntity)[:]
    payload = [UserSchema.from_orm(u).dict() for u in users]

outfile = Path("users.json")
outfile.write_text(json.dumps(payload, indent=2, ensure_ascii=False), encoding="utf-8")
print(outfile.read_text(encoding="utf-8"))


## 7. Dobre praktyki i podsumowanie
- JSON + Pydantic do wymiany i walidacji danych.
- SQLite do prototypów; rozważ migrację do pełnych baz dla większych projektów.
- ORM-y (Pony, SQLAlchemy) upraszczają logikę domenową i relacje.
- Nie ładuj niezaufanych plików Pickle.
- Rozdzielaj walidację (Pydantic) od warstwy trwałości (ORM/baza).


**Podsumowanie:** Zintegrowane podejście do danych (serializacja + walidacja + trwałość) zwiększa bezpieczeństwo i jakość aplikacji.

**Pytanie kontrolne:** Co ryzykujesz, przechowując krytyczne dane tylko w plikach JSON?


## Sprawdź swoją wiedzę
1. Czym różni się `pickle` od `json`?
2. Po co używać `orm_mode=True` w Pydantic?
3. Jakie zalety oferuje ORM w porównaniu z bezpośrednim `sqlite3`?
